In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from mpl_toolkits.mplot3d import Axes3D
import scipy.stats as stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols
from scipy.stats import levene
from scipy.stats import kstest
from statsmodels.stats.anova import anova_lm#方差分析
from statsmodels.stats.multicomp import pairwise_tukeyhsd#两两比较
from scipy.stats import ttest_rel#配对样本t检验
import pingouin as pg
import re
from scipy.optimize import curve_fit
import palettable
from palettable.cartocolors.qualitative import Bold_9

# matplotlib 图形正常显示中文及负号
plt.rcParams['font.sans-serif'] = ['SimHei']   # 用来正常显示中文标签
plt.rcParams['axes.unicode_minus'] = False    # 用来正常显示负号
parameters = {'axes.labelsize': 15,
          'axes.titlesize': 20,'xtick.labelsize':13,'ytick.labelsize':13}
plt.rcParams.update(parameters)
#3d绘图内嵌可旋转
#%matplotlib notebook
#3d绘图单独窗口
%matplotlib qt5
%config InlineBackend.figure_format = 'svg'#矢量图设置，设定显示图片的分辨率

pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [3]:
data=pd.read_csv(r'C:\Users\11054\.spyder-py3\my_project\在线评论\在线评论-2024-9-14\数据-2024-9-14\data_all_processing.csv')
data.columns

Index(['step', '提供方1类型', '提供方2类型', '提供方1利他程度', '提供方2利他程度', 'OP1', 'OP2', 'PCIP1', 'PCIP2', 'ARPP1', 'ARPP2', 'ARSP1', 'ARSP2', '服务人数1', '服务人数2', '总服务人数', '评论1', '评论2', '社会总福利', '平台收益', '提供方收益1', '提供方收益2', '提供方总收益', '消费者收益1', '消费者收益2', '消费者总收益', '消费者满意度1', '消费者满意度2', '消费者总满意度', '消费者剩余1', '消费者剩余2', '消费者总剩余', '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数', '被激励好评数量1', '被激励好评数量2', '获得激励数量1', '获得激励数量2', '好评占比1', '好评占比2', '总好评数量占比', 'PERP1', 'PERP2', 'random_seed', 'PCIP', 'OP', 'PERP', 'ARPP', 'ARSP', '平均毛利率', '平均净利率', '毛利率1', '毛利率2', '净利率1', '净利率2', '消费者平均收益1', '消费者平均收益2', '评论1_当期', '评论2_当期', '评论1_lag1', '评论2_lag1', '总评论数量1', '总评论数量2', '总评论数量', '好评数量1', '好评数量2', '好评总数', '当期好评占比1', '当期好评占比2', '被激励好评占好评比例1', '被激励好评占好评比例2', '被激励好评占好评比例', '获得激励评论数量占评论比例1', '获得激励评论数量占评论比例2', '获得激励评论数量占评论比例', 'RP1', 'RP2', 'RP1_', 'RP2_', 'val1', 'expect1', 'val2', 'expect2', 'market_type'], dtype='object')

In [4]:
columns = ['step', '提供方1类型', '提供方2类型', '提供方1利他程度', '提供方2利他程度', '夸大宣传程度1',
               '夸大宣传程度2', '好评激励1', '好评激励2',
               '价格1', '价格2', '服务质量1', '服务质量2', '服务人数1', '服务人数2', '总服务人数', '评论1', '评论2',
               '社会总福利', '平台收益', '提供方收益1', '提供方收益2', '提供方总收益', '消费者收益1', '消费者收益2',
               '消费者总收益', '消费者满意度1', '消费者满意度2', '消费者总满意度','消费者剩余1', '消费者剩余2', '消费者总剩余',
               '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数', '被激励好评数量1', '被激励好评数量2',
               '获得激励数量1', '获得激励数量2',
               '好评占比1', '好评占比2', '总好评数量占比']

In [24]:
def remove_outliers_3sigma_multi(df, columns):
    df_filtered = df.copy()
    for column in columns:
        mean = df_filtered[column].mean()
        std = df_filtered[column].std()
        
        # Apply the 3-sigma rule to filter out outliers for each column
        df_filtered = df_filtered[(df_filtered[column] >= mean - 6 * std) & (df_filtered[column] <= mean + 6 * std)]
    return df_filtered

data['消费者平均满意度']=(data['消费者满意度1']+data['消费者满意度2'])/(data['服务人数1']+data['服务人数2'])
data['消费者平均收益']=(data['消费者收益1']+data['消费者收益2'])/(data['服务人数1']+data['服务人数2'])
data['消费者平均收益1']=data['消费者收益1']/data['服务人数1']
data['消费者平均收益2']=data['消费者收益2']/data['服务人数2']
data['RP_']=(data['RP1_']+data['RP2_'])/2
df=data[data['market_type']=='hybrid']
columns=['OP1','PCIP1','ARPP1','ARSP1','OP2','PCIP2','ARPP2','ARSP2','OP','PCIP','ARPP','ARSP']#['平台收益','提供方收益1','提供方收益2','提供方总收益','消费者平均收益1','消费者平均收益2','消费者平均收益','消费者平均满意度','社会总福利','总服务人数','RP_']
df=remove_outliers_3sigma_multi(df,columns)
desc = df[columns].describe()
# Combine mean and std into the format 'mean(std)'
for col in columns:
    mean = desc.loc['mean', col]
    std = desc.loc['std', col]
    desc.at['mean(std)', col] = f"{mean:.3f}({std:.3f})"

# Drop the original mean and std rows
desc = desc.drop(['mean', 'std'])
desc

,OP1,PCIP1,ARPP1,ARSP1,OP2,PCIP2,ARPP2,ARSP2,OP,PCIP,ARPP,ARSP
count,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0,891000.0
min,0.0,0.0,0.030947,0.0,0.0,0.0,0.000244,0.0,0.000586,0.00033,0.035891,0.0
25%,0.075311,0.023223,0.364646,0.078313,0.039048,0.068278,0.228102,0.04169,0.092637,0.066081,0.377525,0.122383
50%,0.156117,0.057289,0.540621,0.25148,0.102784,0.134286,0.396997,0.129586,0.137033,0.104615,0.496063,0.211256
75%,0.238022,0.111429,0.716841,0.402612,0.194945,0.211648,0.635476,0.28377,0.179084,0.147253,0.616188,0.309315
max,0.3,0.299853,1.0,0.846731,0.3,0.3,1.0,0.865165,0.298278,0.298755,0.989044,0.728896
mean(std),0.155(0.089),0.076(0.066),0.549(0.215),0.257(0.197),0.120(0.089),0.140(0.085),0.444(0.268),0.185(0.173),0.137(0.063),0.108(0.053),0.496(0.164),0.221(0.128)


In [20]:
anova_d('RP_',data)

RP_的方差齐性检验结果为 LeveneResult(statistic=118314.8455194288, pvalue=0.0)
        Source  ddof1         ddof2             F  p-unc       np2
0  market_type      2  1.644146e+06  3.652698e+06    0.0  0.709179
多重比较：
               A              B   mean(A)   mean(B)      diff        se            T            df  pval    hedges
0         hybrid  pure-altruism  0.891575  1.081590 -0.190014  0.000121 -1572.310600  1.536674e+06   0.0 -2.355669
1         hybrid    pure-egoism  0.891575  0.704368  0.187207  0.000168  1111.621092  1.653047e+06   0.0  1.665454
2  pure-altruism    pure-egoism  1.081590  0.704368  0.377222  0.000150  2513.347172  1.297924e+06   0.0  3.765550


In [9]:
def anova_d(column,data):
    gb=data.groupby('market_type')
    groups=[]
    for a,group in gb:
        groups.append(group[column])
    l=levene(*groups)
    print(column+'的方差齐性检验结果为',l)
    if l.pvalue>0.05:
        formula = column+"~ C(market_type)"
        model=anova_lm(ols(formula,data=data).fit())
        print(model)
        re=pairwise_tukeyhsd(data[column],data['market_type'],alpha=0.05)
        print('多重比较：')
        print(re.summary())
    else:
        model=pg.welch_anova(dv=column, between='market_type', data=data)
        print(model)
        print('多重比较：')
        print(pg.pairwise_gameshowell(dv=column, between='market_type', data=data))
        
def anova_h(column,sm_all):
    sm_2=sm_all[sm_all.market_type=='hybrid']
    gb=sm_2.groupby('s_num')
    groups=[]
    for a,group in gb:
        groups.append(group[column])
    l=levene(*groups)
    print(column+'的方差齐性检验结果为',l)
    if l.pvalue>0.05:
        formula =column+ '~ C(s_num)'
        model=anova_lm(ols(formula,data=sm_2).fit())
        print(model)
        re=pairwise_tukeyhsd(sm_2[column],sm_2['s_num'],alpha=0.05)
        print('多重比较：')
        print(re.summary())
    else:
        model=pg.welch_anova(dv=column, between='s_num', data=sm_2)
        print(model)
        print('多重比较：')
        print(pg.pairwise_gameshowell(dv=column, between='s_num', data=sm_2))
        
def anova_ph(column,sm_2):
    gb=sm_2.groupby('s_num')
    groups=[]
    for a,group in gb:
        groups.append(group[column])
    l=levene(*groups)
    print(column+'的方差齐性检验结果为',l)
    if l.pvalue>0.05:
        formula =column+ '~ C(s_num)'
        model=anova_lm(ols(formula,data=sm_2).fit())
        print(model)
        re=pairwise_tukeyhsd(sm_2[column],sm_2['s_num'],alpha=0.05)
        print('多重比较：')
        print(re.summary())
    else:
        model=pg.welch_anova(dv=column, between='s_num', data=sm_2)
        print(model)
        print('多重比较：')
        print(pg.pairwise_gameshowell(dv=column, between='s_num', data=sm_2))

# 验证与确认

### 不同市场下平均好评激励金额不同

In [12]:
data=pd.read_csv(r'C:\Users\11054\.spyder-py3\my_project\在线评论\在线评论-2024-9-14\数据-2024-9-14\data_all_processing.csv')

In [8]:
data[data['market_type']=='pure-altruism'][['总服务人数']].describe()

,总服务人数
count,891000.000000
mean,252.695627
std,35.481652
min,23.000000
25%,252.000000
50%,264.000000
75%,273.000000
max,315.000000


In [9]:
sns.boxplot(x = 'market_type', y = 'PCIP',data = data, showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='PCIP'>

In [61]:
column='PCIP'
anova_d(column,data)

PCIP的方差齐性检验结果为 LeveneResult(statistic=5965.939812306917, pvalue=0.0)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  233591.611317  70285.030508    0.0  0.238878
多重比较：
               A              B   mean(A)   mean(B)      diff        se           T             df  pval    hedges
0         hybrid  pure-altruism  0.107989  0.142775 -0.034786  0.000228 -152.601074  198079.107659   0.0 -0.601935
1         hybrid    pure-egoism  0.107989  0.075643  0.032346  0.000210  154.218339  158819.530131   0.0  0.661282
2  pure-altruism    pure-egoism  0.142775  0.075643  0.067132  0.000180  373.239838  335987.643261   0.0  1.250398


### 混合市场下，两个提供方间好评激励决策的比较

In [10]:
#方差分析
sm1=pd.DataFrame(data[['PCIP1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['PCIP2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'PCIP1':'PCIP'})
sm2=sm2.rename(columns={'PCIP2':'PCIP'})
sm_all=pd.concat([sm1,sm2])
sns.boxplot(x = 's_num', y = 'PCIP', data = sm_all[sm_all.market_type=='hybrid'], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='s_num', ylabel='PCIP'>

In [11]:
column='PCIP'
anova_h(column,sm_all)

PCIP的方差齐性检验结果为 LeveneResult(statistic=94404.6939402063, pvalue=0.0)
  Source  ddof1         ddof2             F  p-unc       np2
0  s_num      1  1.684468e+06  308893.87316    0.0  0.147733
多重比较：
   A  B   mean(A)   mean(B)    diff        se           T            df  pval    hedges
0  1  2  0.076204  0.139705 -0.0635  0.000114 -555.782217  1.684468e+06   0.0 -0.832685


In [12]:
#两配对样本t检验
sns.violinplot(x = 's_num', y = column, data = sm_all[sm_all.market_type=='hybrid'])

<Axes: xlabel='s_num', ylabel='PCIP'>

In [48]:
#配对样本t检验
print(ttest_rel(data[data['market_type']=='hybrid']['PCIP1'],data[data['market_type']=='hybrid']['PCIP2']))
print(pg.ttest(data[data['market_type']=='hybrid']['PCIP1'],data[data['market_type']=='hybrid']['PCIP2'], paired=True))

TtestResult(statistic=-173.18698076242734, pvalue=0.0, df=89099)
                 T    dof alternative  p-val           CI95%   cohen-d BF10  power
T-test -173.186981  89099   two-sided    0.0  [-0.06, -0.06]  0.831086  inf    1.0


C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)


### 跨市场比较

In [8]:
#方差分析
def cross_market_compare(column_str):
    sm1=pd.DataFrame(data[data['market_type']=='pure-egoism'][[column_str+'1']])
    sm1['s_num']=0
    sm2=pd.DataFrame(data[data['market_type']=='hybrid'][[column_str+'1']])
    sm2['s_num']=1
    sm1=sm1.rename(columns={column_str+'1':column_str})
    sm2=sm2.rename(columns={column_str+'1':column_str})
    sm_all=pd.concat([sm1,sm2])
    column=column_str
    anova_ph(column,sm_all)
    fig,axes=plt.subplots(1,2,figsize=(8,3),sharey=True)
    sns.boxplot(x = 's_num', y = column_str, data = sm_all,ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
    sm1=pd.DataFrame(data[data['market_type']=='hybrid'][[column_str+'2']])
    sm1['s_num']=1
    sm2=pd.DataFrame(data[data['market_type']=='pure-altruism'][[column_str+'2']])
    sm2['s_num']=2
    sm1=sm1.rename(columns={column_str+'2':column_str})
    sm2=sm2.rename(columns={column_str+'2':column_str})
    sm_all=pd.concat([sm1,sm2])
    column=column_str
    anova_ph(column,sm_all)
    sns.boxplot(x = 's_num', y = column_str, data = sm_all,ax=axes[1], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
cross_market_compare('PCIP')

PCIP的方差齐性检验结果为 LeveneResult(statistic=5.210016960912059, pvalue=0.02245992470399829)
  Source  ddof1         ddof2         F     p-unc       np2
0  s_num      1  50265.430093  0.982291  0.321639  0.000013
多重比较：
   A  B   mean(A)   mean(B)      diff       se         T            df      pval    hedges
0  0  1  0.075173  0.075688 -0.000516  0.00052 -0.991106  50265.430093  0.321639 -0.007654
PCIP的方差齐性检验结果为 LeveneResult(statistic=2.224606348197071, pvalue=0.13583170885484874)
               df      sum_sq   mean_sq         F    PR(>F)
C(s_num)      1.0    0.073935  0.073935  10.14104  0.001451
Residual  71101.0  518.372979  0.007291       NaN       NaN
多重比较：
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
     1      2   0.0021 0.0015 0.0008 0.0035   True
--------------------------------------------------


### 不同市场下平均夸大宣传程度

In [13]:
sns.boxplot(x = 'market_type', y = 'OP', data = data, showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='OP'>

In [45]:
column='OP'
anova_d(column,data)

OP的方差齐性检验结果为 LeveneResult(statistic=28.120679786006395, pvalue=6.139242163834169e-13)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  238248.867858  10759.534399    0.0  0.046227
多重比较：
               A              B   mean(A)  mean(B)      diff        se           T             df  pval    hedges
0         hybrid  pure-altruism  0.136942  0.12135  0.015592  0.000258   60.346655  178997.695085   0.0  0.247190
1         hybrid    pure-egoism  0.136942  0.15251 -0.015569  0.000259  -60.112801  180126.016141   0.0 -0.245653
2  pure-altruism    pure-egoism  0.121350  0.15251 -0.031161  0.000212 -146.693091  356380.252653   0.0 -0.491439


### 混合市场下，两个提供方间夸大宣传程度的比较

In [14]:
#方差分析
sm1=pd.DataFrame(data[['OP1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['OP2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'OP1':'OP'})
sm2=sm2.rename(columns={'OP2':'OP'})
sm_all=pd.concat([sm1,sm2])
fig,axes=plt.subplots(1,2,figsize=(8,3),sharey=True)
sns.boxplot(x = 's_num', y = 'OP', data = sm_all[sm_all.market_type=='hybrid'],ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = 'OP', data = sm_all[sm_all.market_type=='hybrid'],ax=axes[1])

<Axes: xlabel='s_num', ylabel='OP'>

In [46]:
column='OP'
anova_h(column,sm_all)
print(ttest_rel(data[data['market_type']=='hybrid']['OP1'],data[data['market_type']=='hybrid']['OP2']))
print(pg.ttest(data[data['market_type']=='hybrid']['OP2'],data[data['market_type']=='hybrid']['OP1'], paired=True))

OP的方差齐性检验结果为 LeveneResult(statistic=52.56683585612653, pvalue=4.1752852927969173e-13)
  Source  ddof1          ddof2            F  p-unc       np2
0  s_num      1  178197.646179  6731.239754    0.0  0.036399
多重比较：
   A  B   mean(A)  mean(B)      diff        se          T             df  pval    hedges
0  1  2  0.154274  0.11961  0.034664  0.000423  82.044133  178197.646179   0.0  0.388707
TtestResult(statistic=81.79497298228681, pvalue=0.0, df=89099)
                T    dof alternative  p-val           CI95%   cohen-d BF10  power
T-test -81.794973  89099   two-sided    0.0  [-0.04, -0.03]  0.388708  inf    1.0


C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)


### 跨市场比较

In [12]:
cross_market_compare('OP')

OP的方差齐性检验结果为 LeveneResult(statistic=0.3554476852375099, pvalue=0.5510472237052046)
               df      sum_sq   mean_sq         F    PR(>F)
C(s_num)      1.0    0.054227  0.054227  6.903581  0.008604
Residual  74387.0  584.303160  0.007855       NaN       NaN
多重比较：
Multiple Comparison of Means - Tukey HSD, FWER=0.05
group1 group2 meandiff p-adj  lower  upper  reject
--------------------------------------------------
     0      1   0.0018 0.0086 0.0005 0.0032   True
--------------------------------------------------
OP的方差齐性检验结果为 LeveneResult(statistic=12.691791780188918, pvalue=0.00036750424488472507)
  Source  ddof1         ddof2         F     p-unc       np2
0  s_num      1  50768.560733  0.526003  0.468296  0.000007
多重比较：
   A  B   mean(A)   mean(B)      diff        se        T            df      pval    hedges
0  1  2  0.119964  0.120477 -0.000513  0.000708 -0.72526  50768.560733  0.468296 -0.005693


### 不同市场下，提供方房间价格服务的差异

In [15]:
fig,axes=plt.subplots(1,2,figsize=(8,4),sharey=True)
sns.boxplot(x = 'market_type', y = 'ARPP', data = data,ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.boxplot(x = 'market_type', y = 'ARSP', data = data,ax=axes[1], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='ARSP'>

In [49]:
column='ARPP'
anova_d(column,data)

ARPP的方差齐性检验结果为 LeveneResult(statistic=2382.070212246824, pvalue=0.0)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  232514.968805  38875.716334    0.0  0.146494
多重比较：
               A              B   mean(A)   mean(B)      diff        se           T             df  pval    hedges
0         hybrid  pure-altruism  0.496502  0.416085  0.080416  0.000675  119.086967  182647.618045   0.0  0.484132
1         hybrid    pure-egoism  0.496502  0.561746 -0.065245  0.000643 -101.439284  159407.333651   0.0 -0.434337
2  pure-altruism    pure-egoism  0.416085  0.561746 -0.145661  0.000523 -278.772622  348042.966146   0.0 -0.933921


In [50]:
column='ARSP'
anova_d(column,data)

ARSP的方差齐性检验结果为 LeveneResult(statistic=657.3460996612935, pvalue=8.682153753411144e-286)
        Source  ddof1          ddof2            F  p-unc       np2
0  market_type      2  236894.387484  3402.669383    0.0  0.015029
多重比较：
               A              B   mean(A)   mean(B)      diff        se          T             df  pval    hedges
0         hybrid  pure-altruism  0.221689  0.204488  0.017201  0.000515  33.380548  170640.216259   0.0  0.139220
1         hybrid    pure-egoism  0.221689  0.239708 -0.018019  0.000531 -33.909952  185219.977953   0.0 -0.137140
2  pure-altruism    pure-egoism  0.204488  0.239708 -0.035220  0.000427 -82.455377  353395.775609   0.0 -0.276235


### 混合市场下，两个提供方房间平均价格和服务质量的比较

In [18]:
#方差分析
sm1=pd.DataFrame(data[['ARPP1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['ARPP2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'ARPP1':'ARPP'})
sm2=sm2.rename(columns={'ARPP2':'ARPP'})
sm_all=pd.concat([sm1,sm2])
sns.boxplot(x = 's_num', y = 'ARPP', data = sm_all[sm_all.market_type=='hybrid'], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
column='ARPP'
anova_h(column,sm_all)

ARPP的方差齐性检验结果为 LeveneResult(statistic=42656.86817696594, pvalue=0.0)
  Source  ddof1         ddof2             F  p-unc       np2
0  s_num      1  1.701641e+06  83777.903729    0.0  0.044902
多重比较：
   A  B   mean(A)   mean(B)      diff        se           T            df  pval    hedges
0  1  2  0.549107  0.443751  0.105356  0.000364  289.444129  1.701641e+06   0.0  0.433651


In [17]:
#方差分析
sm1=pd.DataFrame(data[['ARSP1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['ARSP2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'ARSP1':'ARSP'})
sm2=sm2.rename(columns={'ARSP2':'ARSP'})
sm_all=pd.concat([sm1,sm2])
sns.boxplot(x = 's_num', y = 'ARSP', data = sm_all[sm_all.market_type=='hybrid'], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
column='ARSP'
anova_h(column,sm_all)

ARSP的方差齐性检验结果为 LeveneResult(statistic=33354.50315671973, pvalue=0.0)
  Source  ddof1         ddof2            F  p-unc       np2
0  s_num      1  1.753552e+06  67596.17857    0.0  0.036546
多重比较：
   A  B  mean(A)   mean(B)      diff        se           T            df  pval    hedges
0  1  2  0.25718  0.185063  0.072116  0.000277  259.992651  1.753552e+06   0.0  0.389527


In [21]:
sm1=pd.DataFrame(data[['ARPP1','ARSP1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['ARPP2','ARSP2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'ARPP1':'ARPP'})
sm2=sm2.rename(columns={'ARPP2':'ARPP'})
sm1=sm1.rename(columns={'ARSP1':'ARSP'})
sm2=sm2.rename(columns={'ARSP2':'ARSP'})
sm_all=pd.concat([sm1,sm2])
sm_2=sm_all[sm_all.market_type=='hybrid']
fig,axes=plt.subplots(2,2,figsize=(12,5))
sns.boxplot(x = 's_num', y = 'ARPP', data = sm_2,ax=axes[0][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = 'ARPP', data = sm_2,ax=axes[0][1])
sns.boxplot(x = 's_num', y = 'ARSP', data = sm_2,ax=axes[1][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = 'ARSP', data = sm_2,ax=axes[1][1])

<Axes: xlabel='s_num', ylabel='ARSP'>

In [31]:
print(ttest_rel(data[data['market_type']=='hybrid']['ARPP1'],data[data['market_type']=='hybrid']['ARPP2']))
print(ttest_rel(data[data['market_type']=='hybrid']['ARSP1'],data[data['market_type']=='hybrid']['ARSP2']))

TtestResult(statistic=40.49854956922508, pvalue=0.0, df=26729)
TtestResult(statistic=33.06691354589958, pvalue=4.846857462056051e-235, df=26729)


### 跨市场比较

In [19]:
cross_market_compare('ARPP')

ARPP的方差齐性检验结果为 LeveneResult(statistic=10.485733072545555, pvalue=0.001203525931993176)
  Source  ddof1         ddof2          F         p-unc      np2
0  s_num      1  48922.272183  77.984911  1.071895e-18  0.00105
多重比较：
   A  B   mean(A)   mean(B)      diff        se         T            df          pval    hedges
0  0  1  0.563918  0.549181  0.014737  0.001669  8.830907  48922.272183  3.778200e-12  0.068898
ARPP的方差齐性检验结果为 LeveneResult(statistic=107.32161765623754, pvalue=3.9440599641009145e-25)
  Source  ddof1        ddof2           F         p-unc       np2
0  s_num      1  47828.70286  153.571172  3.255085e-35  0.002231
多重比较：
   A  B   mean(A)   mean(B)      diff        se          T           df  pval    hedges
0  1  2  0.443014  0.417458  0.025555  0.002062  12.392384  47828.70286   0.0  0.099359


In [20]:
cross_market_compare('ARSP')

ARSP的方差齐性检验结果为 LeveneResult(statistic=229.85770182119316, pvalue=7.655995996224218e-52)
  Source  ddof1         ddof2           F         p-unc       np2
0  s_num      1  46751.533183  110.396562  8.571834e-26  0.001539
多重比较：
   A  B   mean(A)   mean(B)      diff       se          T            df  pval    hedges
0  0  1  0.241688  0.257549 -0.015862  0.00151 -10.506977  46751.533183   0.0 -0.083408
ARSP的方差齐性检验结果为 LeveneResult(statistic=230.93465536291413, pvalue=4.502734150424549e-52)
  Source  ddof1         ddof2           F         p-unc       np2
0  s_num      1  53439.107313  218.924155  1.945594e-49  0.002937
多重比较：
   A  B   mean(A)   mean(B)      diff        se          T            df          pval    hedges
0  1  2  0.184787  0.205555 -0.020768  0.001404 -14.796086  53439.107313  3.315903e-11 -0.114053


### 不同市场下，消费者总满意度和消费者总剩余的差异

In [22]:
fig,axes=plt.subplots(1,2,figsize=(8,4),sharey=True)
sns.boxplot(x = 'market_type', y = '消费者总满意度', data = data,ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.boxplot(x = 'market_type', y = '消费者总剩余', data = data,ax=axes[1], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='消费者总剩余'>

In [26]:
plt.figure(figsize=(8, 6))
sns.boxplot(x = 'market_type', y = '消费者总剩余', data = data, showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='消费者总剩余'>

### 混合市场下，两个提供方消费者剩余的比较

In [24]:
#方差分析
sm1=pd.DataFrame(data[['消费者剩余1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['消费者剩余2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'消费者剩余1':'消费者剩余'})
sm2=sm2.rename(columns={'消费者剩余2':'消费者剩余'})
sm_all=pd.concat([sm1,sm2])
fig,axes=plt.subplots(1,2,figsize=(8,3),sharey=True)
sns.boxplot(x = 's_num', y = '消费者剩余', data = sm_all[sm_all.market_type=='hybrid'],ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = '消费者剩余', data = sm_all[sm_all.market_type=='hybrid'],ax=axes[1])

<Axes: xlabel='s_num', ylabel='消费者剩余'>

In [19]:
column='消费者剩余'
anova_h(column,sm_all)
print(ttest_rel(data[data['market_type']=='hybrid']['消费者剩余1'],data[data['market_type']=='hybrid']['消费者剩余2']))
print(pg.ttest(data[data['market_type']=='hybrid']['消费者剩余1'],data[data['market_type']=='hybrid']['消费者剩余2'], paired=True))

消费者剩余的方差齐性检验结果为 LeveneResult(statistic=152642.03388612586, pvalue=0.0)
  Source  ddof1         ddof2             F  p-unc       np2
0  s_num      1  106859.13938  80235.469785    0.0  0.310469
多重比较：
   A  B    mean(A)    mean(B)       diff        se           T            df  pval    hedges
0  1  2  16.557004  59.750199 -43.193195  0.152487 -283.258662  106859.13938   0.0 -1.342015
TtestResult(statistic=-284.6428367498738, pvalue=0.0, df=89099)
                 T    dof alternative  p-val            CI95%   cohen-d BF10  power
T-test -284.642837  89099   two-sided    0.0  [-43.49, -42.9]  1.342021  inf    1.0


C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)


### 跨市场比较

In [24]:
cross_market_compare('消费者剩余')

消费者剩余的方差齐性检验结果为 LeveneResult(statistic=83.97142073629857, pvalue=5.14296260883047e-20)
  Source  ddof1         ddof2           F         p-unc       np2
0  s_num      1  50145.119778  185.864786  3.022909e-42  0.002454
多重比较：
   A  B   mean(A)   mean(B)    diff       se          T            df          pval    hedges
0  0  1  8.106049  9.475649 -1.3696  0.10046 -13.633224  50145.119778  1.649403e-11 -0.105378
消费者剩余的方差齐性检验结果为 LeveneResult(statistic=254.1942443449269, pvalue=3.974606301204854e-57)
  Source  ddof1         ddof2          F         p-unc       np2
0  s_num      1  46679.258035  150.66808  1.400621e-34  0.002227
多重比较：
   A  B    mean(A)    mean(B)      diff      se          T            df  pval    hedges
0  1  2  45.378597  49.348229 -3.969632  0.3234 -12.274693  46679.258035   0.0 -0.099273


### 不同市场下，提供方房间毛利率与净利率的差异

In [27]:
fig,axes=plt.subplots(1,2,figsize=(8,4),sharey=True)
sns.boxplot(x = 'market_type', y = '平均毛利率', data = data,ax=axes[0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.boxplot(x = 'market_type', y = '平均净利率', data = data,ax=axes[1], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})

<Axes: xlabel='market_type', ylabel='平均净利率'>

In [12]:
column='平均毛利率'
anova_d(column,data)

平均毛利率的方差齐性检验结果为 LeveneResult(statistic=104.32370033451555, pvalue=2.290286978178935e-44)
        Source  ddof1       ddof2            F  p-unc       np2
0  market_type      2  1776.28078  3246.963482    0.0  0.697879
多重比较：
     A    B  mean(A)   mean(B)      diff        se          T           df          pval    hedges
0  0.0  1.0  0.32079  0.283450  0.037340  0.001889  19.769135  1703.266281  0.000000e+00  0.931537
1  0.0  2.0  0.32079  0.174094  0.146696  0.001842  79.636773  1731.109662  1.635359e-13  3.752547
2  1.0  2.0  0.28345  0.174094  0.109356  0.002058  53.142145  1794.956968  0.000000e+00  2.504100


In [13]:
column='平均净利率'
anova_d(column,data)

平均净利率的方差齐性检验结果为 LeveneResult(statistic=29.69127209832442, pvalue=1.758542153128032e-13)
        Source  ddof1        ddof2            F  p-unc      np2
0  market_type      2  1773.166377  4755.687204    0.0  0.74763
多重比较：
     A    B   mean(A)   mean(B)      diff        se          T           df          pval    hedges
0  0.0  1.0  0.234854  0.198227  0.036627  0.001728  21.190615  1772.543398  0.000000e+00  0.998518
1  0.0  2.0  0.234854  0.095284  0.139570  0.001503  92.847985  1751.089657  0.000000e+00  4.375070
2  1.0  2.0  0.198227  0.095284  0.102943  0.001618  63.626960  1668.960312  6.357137e-13  2.998152


### 混合市场下，两个提供方房间平均毛利率与净利率的比较

In [26]:
sm1=pd.DataFrame(data[['毛利率1','净利率1','market_type']])
sm1['s_num']=1
sm2=pd.DataFrame(data[['毛利率2','净利率2','market_type']])
sm2['s_num']=2
sm1=sm1.rename(columns={'毛利率1':'平均毛利率'})
sm2=sm2.rename(columns={'毛利率2':'平均毛利率'})
sm1=sm1.rename(columns={'净利率1':'平均净利率'})
sm2=sm2.rename(columns={'净利率2':'平均净利率'})
sm_all=pd.concat([sm1,sm2])
sm_2=sm_all[sm_all.market_type=='hybrid']
fig,axes=plt.subplots(2,2,figsize=(12,5))
sns.boxplot(x = 's_num', y = '平均毛利率', data = sm_2,ax=axes[0][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = '平均毛利率', data = sm_2,ax=axes[0][1])
sns.boxplot(x = 's_num', y = '平均净利率', data = sm_2,ax=axes[1][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
sns.violinplot(x = 's_num', y = '平均净利率', data = sm_2,ax=axes[1][1])

<Axes: xlabel='s_num', ylabel='平均净利率'>

In [15]:
column='平均毛利率'
anova_h(column,sm_all)
print('\n'*3)
column='平均净利率'
anova_h(column,sm_all)

平均毛利率的方差齐性检验结果为 LeveneResult(statistic=89.26234102369405, pvalue=1.0352622994500837e-20)
  Source  ddof1        ddof2          F          p-unc       np2
0  s_num      1  1576.044177  566.55979  3.089546e-107  0.239605
多重比较：
   A  B   mean(A)   mean(B)      diff        se          T           df  pval    hedges
0  1  2  0.253327  0.313573 -0.060246  0.002531 -23.802516  1576.044177   0.0 -1.121593




平均净利率的方差齐性检验结果为 LeveneResult(statistic=84.87341524733047, pvalue=8.595661460894557e-20)
  Source  ddof1        ddof2           F         p-unc       np2
0  s_num      1  1561.241165  464.782319  1.894851e-90  0.205403
多重比较：
   A  B   mean(A)   mean(B)      diff        se          T           df  pval    hedges
0  1  2  0.175193  0.221262 -0.046069  0.002137 -21.558811  1561.241165   0.0 -1.015868


### 跨市场比较

In [27]:
cross_market_compare('毛利率')

毛利率的方差齐性检验结果为 LeveneResult(statistic=628.4335783871683, pvalue=4.1165454355852394e-138)
  Source  ddof1         ddof2            F  p-unc       np2
0  s_num      1  67506.652231  1625.374208    0.0  0.016714
多重比较：
   A  B  mean(A)   mean(B)      diff        se         T            df  pval    hedges
0  0  1  0.32223  0.291632  0.030598  0.000759  40.31593  67506.652231   0.0  0.277007
毛利率的方差齐性检验结果为 LeveneResult(statistic=1625.4817011881964, pvalue=0.0)
  Source  ddof1         ddof2            F          p-unc       np2
0  s_num      1  39329.384276  1246.931276  6.258357e-269  0.020539
多重比较：
   A  B   mean(A)   mean(B)      diff        se          T            df  pval    hedges
0  1  2  0.258227  0.211904  0.046323  0.001312  35.311914  39329.384276   0.0  0.304302


In [28]:
cross_market_compare('净利率')

净利率的方差齐性检验结果为 LeveneResult(statistic=689.8965276119242, pvalue=2.318609092326677e-151)
  Source  ddof1         ddof2            F  p-unc       np2
0  s_num      1  66594.251603  1691.579467    0.0  0.017603
多重比较：
   A  B   mean(A)   mean(B)      diff       se          T            df          pval   hedges
0  0  1  0.237643  0.209255  0.028388  0.00069  41.128816  66594.251603  2.747047e-11  0.28441
净利率的方差齐性检验结果为 LeveneResult(statistic=1721.0940104267672, pvalue=0.0)
  Source  ddof1         ddof2            F          p-unc       np2
0  s_num      1  38900.471323  1484.623753  8.742689e-319  0.024563
多重比较：
   A  B   mean(A)   mean(B)     diff        se          T            df  pval    hedges
0  1  2  0.191775  0.149285  0.04249  0.001103  38.530816  38900.471323   0.0  0.333461


### 不同市场下各利益相关者收益的比较

In [28]:
columns=['社会总福利','平台收益','提供方总收益','消费者总收益']
fig,axes=plt.subplots(4,2,figsize=(10,20))
for i,column in zip(range(4),columns):
    sns.boxplot(x = 'market_type', y = column, data = data,ax=axes[i][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
    sns.violinplot(x = 'market_type', y = column, data = data,ax=axes[i][1])
#fig.savefig('1.png')

In [52]:
columns=['社会总福利','平台收益','提供方总收益','消费者总收益']
for column in columns:
    anova_d(column,data)
    print('\n'*3)

社会总福利的方差齐性检验结果为 LeveneResult(statistic=8772.784260875307, pvalue=0.0)
        Source  ddof1          ddof2            F  p-unc       np2
0  market_type      2  228595.892523  4830.222271    0.0  0.019996
多重比较：
               A              B    mean(A)     mean(B)       diff        se          T             df  pval    hedges
0         hybrid  pure-altruism  232.07342  240.030860  -7.957440  0.155451 -51.189279  149665.071665   0.0 -0.224775
1         hybrid    pure-egoism  232.07342  227.732975   4.340445  0.169594  25.593199  191032.537765   0.0  0.102322
2  pure-altruism    pure-egoism  240.03086  227.732975  12.297885  0.128914  95.396248  331067.036040   0.0  0.319589




平台收益的方差齐性检验结果为 LeveneResult(statistic=1343.6665185586498, pvalue=0.0)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  238121.721811  20113.335661    0.0  0.083347
多重比较：
               A              B    mean(A)    mean(B)       diff        se           T             df 

In [53]:
columns=['提供方收益1','提供方收益2','消费者平均收益1','消费者平均收益2']
for column in columns:
    anova_d(column,data)
    print('\n'*3)

提供方收益1的方差齐性检验结果为 LeveneResult(statistic=1977.7019232491386, pvalue=0.0)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  241719.728201  77988.648895    0.0  0.243508
多重比较：
               A              B    mean(A)    mean(B)       diff        se           T             df  pval    hedges
0         hybrid  pure-altruism  71.421254  38.971341  32.449913  0.102863  315.467361  174795.613529   0.0  1.303791
1         hybrid    pure-egoism  71.421254  70.111593   1.309661  0.111269   11.770210  210595.012921   0.0  0.045369
2  pure-altruism    pure-egoism  38.971341  70.111593 -31.140252  0.093010 -334.805681  341608.035408   0.0 -1.121639




提供方收益2的方差齐性检验结果为 LeveneResult(statistic=2029.722909986046, pvalue=0.0)
        Source  ddof1          ddof2             F  p-unc       np2
0  market_type      2  233573.343675  63356.496353    0.0  0.232318
多重比较：
               A              B    mean(A)    mean(B)       diff        se           T           

### 混合市场下各方收益的比较

In [54]:
columns=[ '提供方收益','消费者收益','消费者平均收益']
columns1=['提供方收益1','消费者收益1','消费者平均收益1','market_type']
columns2=['提供方收益2','消费者收益2','消费者平均收益2','market_type']

sm1=pd.DataFrame(data[columns1])
sm1['s_num']=1
sm2=pd.DataFrame(data[columns2])
sm2['s_num']=2
for c,c1,c2 in zip(columns,columns1,columns2):
    sm1=sm1.rename(columns={c1:c})
    sm2=sm2.rename(columns={c2:c})
    #两配对样本t检验
    print(ttest_rel(data[data['market_type']=='hybrid'][c1],data[data['market_type']=='hybrid'][c2]))
    print(pg.ttest(data[data['market_type']=='hybrid'][c1],data[data['market_type']=='hybrid'][c2], paired=True))
sm_all=pd.concat([sm1,sm2])
sm_2=sm_all[sm_all.market_type=='hybrid']
fig,axes=plt.subplots(3,2,figsize=(10,5))
for i,c in enumerate(columns):
    sns.boxplot(x = 's_num', y = c, data = sm_2,ax=axes[i][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
    sns.violinplot(x = 's_num', y = c, data = sm_2,ax=axes[i][1])
for column in columns:
    anova_h(column,sm_2)
    print('\n'*3)


TtestResult(statistic=246.52969816049685, pvalue=0.0, df=89099)
                 T    dof alternative  p-val          CI95%   cohen-d BF10  power
T-test  246.529698  89099   two-sided    0.0  [32.48, 33.0]  1.227083  inf    1.0
TtestResult(statistic=-268.4441524370032, pvalue=0.0, df=89099)


C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)
C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)


                 T    dof alternative  p-val            CI95%   cohen-d BF10  power
T-test -268.444152  89099   two-sided    0.0  [-38.1, -37.55]  1.206056  inf    1.0
TtestResult(statistic=nan, pvalue=nan, df=nan)
                 T    dof alternative  p-val           CI95%   cohen-d BF10  power
T-test -313.756234  87681   two-sided    0.0  [-0.31, -0.31]  1.412622  inf    1.0


C:\D\anaconda\Lib\site-packages\pingouin\bayesian.py:152: RuntimeWarning: divide by zero encountered in scalar divide
  bf10 = 1 / ((1 + t**2 / df) ** (-(df + 1) / 2) / integr)


提供方收益的方差齐性检验结果为 LeveneResult(statistic=4935.679657584856, pvalue=0.0)
  Source  ddof1          ddof2             F  p-unc       np2
0  s_num      1  176281.984375  67080.391966    0.0  0.273487
多重比较：
   A  B    mean(A)    mean(B)       diff        se           T             df  pval    hedges
0  1  2  71.421254  38.684403  32.736851  0.126398  258.998826  176281.984375   0.0  1.227078




消费者收益的方差齐性检验结果为 LeveneResult(statistic=70682.30326347938, pvalue=0.0)
  Source  ddof1          ddof2             F  p-unc       np2
0  s_num      1  112832.106741  64801.094029    0.0  0.266672
多重比较：
   A  B    mean(A)    mean(B)       diff        se          T             df  pval   hedges
0  1  2  15.332299  53.156252 -37.823953  0.148585 -254.56059  112832.106741   0.0 -1.20605




消费者平均收益的方差齐性检验结果为 LeveneResult(statistic=nan, pvalue=nan)
  Source  ddof1          ddof2             F  p-unc       np2
0  s_num      1  126638.874661  88446.306074    0.0  0.335282
多重比较：
   A  B   mean(A)  mean(B)      

### 跨平台比较

In [31]:
columns=['提供方收益','消费者收益','消费者平均收益']
for column_str in columns:
    cross_market_compare(column_str)

提供方收益的方差齐性检验结果为 LeveneResult(statistic=724.5470198561105, pvalue=7.967449198292006e-159)
  Source  ddof1         ddof2          F         p-unc       np2
0  s_num      1  58280.218278  46.123243  1.121046e-11  0.000545
多重比较：
   A  B   mean(A)    mean(B)      diff        se         T            df          pval    hedges
0  0  1  70.06044  71.492385 -1.431945  0.210846 -6.791409  58280.218278  3.409117e-11 -0.049608
提供方收益的方差齐性检验结果为 LeveneResult(statistic=1448.578991899516, pvalue=8.551599686e-314)
  Source  ddof1         ddof2         F     p-unc       np2
0  s_num      1  44756.229385  0.542503  0.461401  0.000008
多重比较：
   A  B    mean(A)    mean(B)      diff        se         T            df      pval    hedges
0  1  2  38.958029  39.114425 -0.156396  0.212336 -0.736548  44756.229385  0.461401 -0.006048
消费者收益的方差齐性检验结果为 LeveneResult(statistic=63.188595984803335, pvalue=1.9044905715450293e-15)
  Source  ddof1         ddof2           F         p-unc       np2
0  s_num      1  49228.05900

### 不同市场下其他指标的比较

In [32]:
'服务人数1', '服务人数2', '总服务人数', '实际到达人数', '办理入住数1', '办理入住数2', '办理入住总数',
'被激励好评数量1', '被激励好评数量2', '获得激励数量1', '获得激励数量2', '好评占比1', '好评占比2', '总好评数量占比', '当期好评占比1', '当期好评占比2',
'总评论数量1', '总评论数量2', '总评论数量', '好评数量1', '好评数量2', '好评总数', 
'被激励好评占好评比例1', '被激励好评占好评比例2', '被激励好评占好评比例',
'获得激励评论数量占评论比例1', '获得激励评论数量占评论比例2', '获得激励评论数量占评论比例'

('获得激励评论数量占评论比例1', '获得激励评论数量占评论比例2', '获得激励评论数量占评论比例')

In [33]:
columns=['总服务人数','实际到达人数','办理入住总数','总好评数量占比','总评论数量', '好评总数', '被激励好评占好评比例', '获得激励评论数量占评论比例']
fig,axes=plt.subplots(8,2,figsize=(10,20))
for i,column in zip(range(8),columns):
    sns.boxplot(x = 'market_type', y = column, data = data,ax=axes[i][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
    sns.violinplot(x = 'market_type', y = column, data = data,ax=axes[i][1])
#fig.savefig('1.png')

In [55]:
columns=['总服务人数','办理入住总数','总好评数量占比','总评论数量', '好评总数', '被激励好评占好评比例', '获得激励评论数量占评论比例']
columns1=['服务人数1','办理入住数1','好评占比1','总评论数量1', '好评数量1', '被激励好评占好评比例1', '获得激励评论数量占评论比例1','market_type']
columns2=['服务人数2','办理入住数2','好评占比2','总评论数量2', '好评数量2', '被激励好评占好评比例2', '获得激励评论数量占评论比例2','market_type']

sm1=pd.DataFrame(data[columns1])
sm1['s_num']=1
sm2=pd.DataFrame(data[columns2])
sm2['s_num']=2
for c,c1,c2 in zip(columns,columns1,columns2):
    sm1=sm1.rename(columns={c1:c})
    sm2=sm2.rename(columns={c2:c})
    print(ttest_rel(data[data['market_type']=='hybrid'][c1],data[data['market_type']=='hybrid'][c2]))
sm_all=pd.concat([sm1,sm2])
sm_2=sm_all[sm_all.market_type=='hybrid']
fig,axes=plt.subplots(7,2,figsize=(10,20))
for i,c in enumerate(columns):
    sns.boxplot(x = 's_num', y = c, data = sm_2,ax=axes[i][0], showmeans=True,meanline=True, meanprops = {'linestyle':'--','color':'red'})
    sns.violinplot(x = 's_num', y = c, data = sm_2,ax=axes[i][1])

TtestResult(statistic=11.2228063943589, pvalue=3.29670074353027e-29, df=89099)
TtestResult(statistic=10.007895724556082, pvalue=1.4480850059597846e-23, df=89099)
TtestResult(statistic=-519.5191620886777, pvalue=0.0, df=89099)
TtestResult(statistic=91.76792336591089, pvalue=0.0, df=89099)
TtestResult(statistic=14.555365102416749, pvalue=6.130433366376347e-48, df=89099)
TtestResult(statistic=nan, pvalue=nan, df=nan)
TtestResult(statistic=nan, pvalue=nan, df=nan)


### 跨市场比较

In [34]:
columns=['服务人数','办理入住数','好评占比','总评论数量', '好评数量', '被激励好评占好评比例', '获得激励评论数量占评论比例']
for column_str in columns:
    cross_market_compare(column_str)

服务人数的方差齐性检验结果为 LeveneResult(statistic=1661.1610453405394, pvalue=0.0)
  Source  ddof1         ddof2           F         p-unc       np2
0  s_num      1  65952.007091  254.205572  4.023089e-57  0.002709
多重比较：
   A  B     mean(A)     mean(B)      diff        se          T            df  pval    hedges
0  0  1  117.174693  121.975608 -4.800914  0.301114 -15.943826  65952.007091   0.0 -0.110741
服务人数的方差齐性检验结果为 LeveneResult(statistic=1218.9682746044732, pvalue=8.129622007484651e-265)
  Source  ddof1        ddof2           F         p-unc       np2
0  s_num      1  37453.81633  375.641325  2.834169e-83  0.006523
多重比较：
   A  B     mean(A)     mean(B)      diff        se          T           df  pval    hedges
0  1  2  119.183652  126.395403 -7.211751  0.372095 -19.381469  37453.81633   0.0 -0.170271
办理入住数的方差齐性检验结果为 LeveneResult(statistic=1704.687561087668, pvalue=0.0)
  Source  ddof1         ddof2          F         p-unc       np2
0  s_num      1  66373.814766  203.35171  4.535354e-46  0.0021